In [39]:
import requests
import rasterio
from rasterio.transform import from_bounds
from PIL import Image
import numpy as np
import json
from pyproj import Proj, transformer
from pyproj import Transformer


bbox = (229674.468, 468666.620, 235316.890, 478309.041)
max_size = 101  # max width or height in pixels
sample_size = 800

# aspect ratio
bbox_width = bbox[2] - bbox[0]
bbox_height = bbox[3] - bbox[1]
aspect_ratio = bbox_width / bbox_height

# largest side gets the max size
if aspect_ratio > 1:
    width = max_size
    height = int(round(max_size / aspect_ratio))
else:
    height = max_size
    width = int(round(max_size * aspect_ratio))

# Grid resolution
steps = max(5, min(30, int(min(bbox_width, bbox_height) // sample_size)))

x_steps = steps
y_steps = steps
x_interval = width / x_steps
y_interval = height / y_steps

print(f"Image size: {width} x {height}, sampling {x_steps} x {y_steps} points")

# Common WMS URL
url = "https://service.pdok.nl/bzk/brobhrpkenset/wms/v1_0"

# GetFeatureInfo parameters
base_params = {
    "SERVICE": "WMS",
    "VERSION": "1.3.0",
    "REQUEST": "GetFeatureInfo",
    "LAYERS": "bhr_kenset",
    "QUERY_LAYERS": "bhr_kenset",
    "INFO_FORMAT": "application/json",
    "FEATURE_COUNT": 1000,
    "WIDTH": width,
    "HEIGHT": height,
    "CRS": "EPSG:28992",
    "BBOX": ",".join(map(str, bbox)),
    "FORMAT": "image/png",
    "TRANSPARENT": "true",
    "STYLES": "",
    "DPI": 135,
    "MAP_RESOLUTION": 135
}

# --- Collect unique features ---
features_seen = {}
features = []

for i in range(x_steps):
    for j in range(y_steps):
        I = int(i * x_interval + x_interval / 2)
        J = int(j * y_interval + y_interval / 2)

        params = base_params.copy()
        params.update({"I": I, "J": J})

        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            for feature in data.get("features", []):
                fid = feature.get("id") or json.dumps(feature.get("geometry"))
                if fid not in features_seen:
                    features_seen[fid] = True
                    features.append(feature)

        except Exception as e:
            print(f"Request failed at I={I}, J={J}: {e}")

print(f"Collected {len(features)} unique features.")
# Save GeoJSON
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

print(json.dumps(geojson_data['features'][0], indent=2))
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)

for feature in geojson_data['features']:
    x, y = feature['geometry']['coordinates']
    lon, lat = transformer.transform(x, y)
    feature['geometry']['coordinates'] = [lon, lat]

output_path = "../downloads/sample_features.geojson"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, indent=2)

print(f"Saved GeoJSON with {len(features)} features to {output_path}")



# ========== GetMap part ===========
# GetMap parameters for image request
params_wms = {
    "SERVICE": "WMS",
    "VERSION": "1.3.0",
    "REQUEST": "GetMap",
    "LAYERS": "bhr_kenset",
    "FORMAT": "image/png",
    "TRANSPARENT": "true",
    "STYLES": "",
    "CRS": "EPSG:28992",
    "BBOX": ",".join(map(str, bbox)),
    "WIDTH": width,
    "HEIGHT": height,
    "DPI": 135,
    "MAP_RESOLUTION": 135
}


response2 = requests.get(url, params=params_wms)
response2.raise_for_status()

with open("../output/output.png", "wb") as f:
    f.write(response2.content)

# Convert PNG to GeoTIFF
img = Image.open("../output/output.png").convert("RGBA")
arr = np.array(img)
transform = from_bounds(*bbox, width=width, height=height)

with rasterio.open(
    "../output/output_georeferenced.tif",
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=4,
    dtype=arr.dtype,
    crs="EPSG:28992",
    transform=transform
) as dst:
    for i in range(4):
        dst.write(arr[:, :, i], i + 1)


Image size: 59 x 101, sampling 7 x 7 points
Collected 1547 unique features.
{
  "id": "bhr_kenset.c25b5527-c956-42de-b198-a8312ff692ca",
  "type": "Feature",
  "bbox": [
    229588.0008076543,
    477205.000578457,
    229588.0008076543,
    477205.000578457
  ],
  "geometry": {
    "type": "Point",
    "coordinates": [
      229588.0008076543,
      477205.000578457
    ]
  },
  "properties": {
    "begindiepte": "0",
    "beschrijfmethode": "AlterraTD19A",
    "boorgat_bemeten": "nee",
    "boormonsters_beschreven": "ja",
    "boormonsters_geanalyseerd": "nee",
    "boormonsters_gefotografeerd": "nee",
    "boortype": "edelmanboor",
    "bro_id": "BHR000000034519",
    "bronhouder": "27378529",
    "einddatum_boring": "2000-03",
    "einddiepte": "1.8",
    "fractieverdeling_bepaald": "nee",
    "in_onderzoek": "nee",
    "kader_inwinning": "gebiedsinrichting",
    "kwaliteitsregime": "IMBRO/A",
    "lokaal_verticaal_referentiepunt": "maaiveld",
    "rapportagedatum_onderzoek": "2017